In [0]:
src_schema='silver'
src_tbl='bookings'
trg_schema='gold'
fact_src=f'Project.{src_schema}.{src_tbl}'
Final_fact=f'Project.{trg_schema}.final_fact'
cdc='modified_date'
backdated_refresh = ''
fact_key_cols = ["dim_passengersKey","dim_flightsKey","dim_airportsKey","booking_date"]

In [0]:
dimension=[
    {
        'table_name': 'Project.gold.dim_passengers',
        'alias': 'passengers',
        'key_col':[('passenger_id','passenger_id')]
    },
    {
        'table_name': 'Project.gold.dim_flights',
        'alias': 'flights',
        'key_col':[('flight_id','flight_id')]
    },
    {
        'table_name': 'Project.gold.dim_Airports',
        'alias': 'airports',
        'key_col':[('airport_id','airport_id')]
    },


]
fact_col=['amount','modified_date','booking_date']

In [0]:
# No Back Dated Refresh
if len(backdated_refresh) == 0:
  
  # If Table Exists In The Destination
  if spark.catalog.tableExists(Final_fact):

    last_load = spark.sql(f"SELECT max({cdc}) FROM {Final_fact}").collect()[0][0]
    
  else:
    last_load = "1900-01-01 00:00:00"

# Yes Back Dated Refresh
else:
  last_load = backdated_refresh

# Test The Last Load 
last_load

In [0]:
def fact_query_generator(
    fact_src,
    fact_col,
    dimensions,
    cdc,
    last_load
):
    fact_alias = 'f'
    fact_col = [f'{fact_alias}.{col}' for col in fact_col]
    clause = []

    for dim in dimensions:
        full_table = dim['table_name']
        alias = dim['alias']
        surrogate_key = f'{alias}.dim_{alias}key'
        fact_col.append(surrogate_key)

        for fk, dk in dim["key_col"]:
            join_clause = f"LEFT JOIN {full_table} AS {alias} ON {alias}.{fk} = {fact_alias}.{dk}"
            clause.append(join_clause)

    select_cols = ',\n'.join(fact_col)
    joined_condition = '\n'.join(clause)
    where_clause = f"{fact_alias}.{cdc} >= '{last_load}'"

    query = f"""
SELECT
    {select_cols}
FROM {fact_src} {fact_alias}
{joined_condition}
WHERE {where_clause}
""".strip()

    return query

In [0]:
q=fact_query_generator(fact_src,fact_col,dimension,cdc,last_load)

In [0]:
# from pyspark.sql.functions import distinct
df_fact = spark.sql(q)


In [0]:
on_cond=[f'src.{col} = trg.{col}' for col in fact_key_cols]
x=' AND '.join(on_cond)

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists(Final_fact):

    dlt_obj = DeltaTable.forName(spark,Final_fact)
    dlt_obj.alias("trg").merge(df_fact.alias("src"),x)\
                        .whenMatchedUpdateAll(condition = f"src.{cdc} >= trg.{cdc}")\
                        .whenNotMatchedInsertAll()\
                        .execute()

else: 

    df_fact.write.format("delta")\
            .mode("append")\
            .saveAsTable(Final_fact)
